In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
from scipy.stats import linregress

from config import weather_api_key
from citipy import citipy
from datetime import datetime


In [8]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [4]:
coordinates = list(lat_lngs)


In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


741

In [9]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(
            city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | airai
Processing Record 2 of Set 1 | mahadday weyne
City not found. Skipping...
Processing Record 3 of Set 1 | tumannyy
City not found. Skipping...
Processing Record 4 of Set 1 | inhambane
Processing Record 5 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 6 of Set 1 | mar del plata
Processing Record 7 of Set 1 | necochea
Processing Record 8 of Set 1 | butaritari
Processing Record 9 of Set 1 | dikson
Processing Record 10 of Set 1 | saskylakh
Processing Record 11 of Set 1 | mataura
Processing Record 12 of Set 1 | faanui
Processing Record 13 of Set 1 | port-gentil
Processing Record 14 of Set 1 | ulladulla
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | kajaani
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | matara
Processing Record 19 of Set 1 | soverato
Processing Record 20 of Set 1 | ende
Processing Record 21 of Set 1 | atuona
Pro

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.9266,125.4092,80.89,34,45,3.15,TL,2021-10-04 03:42:02
1,Inhambane,-23.8650,35.3833,73.38,94,75,9.22,MZ,2021-10-04 03:38:57
2,Mar Del Plata,-38.0023,-57.5575,46.13,68,0,7.00,AR,2021-10-04 03:32:40
3,Necochea,-38.5473,-58.7368,41.59,84,0,7.63,AR,2021-10-04 03:42:03
4,Butaritari,3.0707,172.7902,82.81,69,7,16.44,KI,2021-10-04 03:42:03
5,Dikson,73.5069,80.5464,35.60,98,100,20.02,RU,2021-10-04 03:42:04
6,Saskylakh,71.9167,114.0833,22.69,90,100,14.97,RU,2021-10-04 03:42:04
7,Mataura,-46.1927,168.8643,52.48,80,100,11.90,NZ,2021-10-04 03:32:14
8,Faanui,-16.4833,-151.7500,78.82,72,14,10.31,PF,2021-10-04 03:32:32
9,Port-Gentil,-0.7193,8.7815,77.36,84,99,12.59,GA,2021-10-04 03:42:05


In [11]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
